In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# **Lightweight Fine-Tuning Project**\n",
    "\n",
    "* **PEFT technique**: LoRA (Low-Rank Adaptation)\n",
    "* **Model**: DistilBERT (distilbert-base-uncased)\n",
    "* **Evaluation approach**: Evaluating accuracy and F1 score on validation set using Hugging Face Trainer\n",
    "* **Fine-tuning dataset**: SST-2 (Stanford Sentiment Treebank) from GLUE benchmark"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **Loading and Evaluating a Foundation Model**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import os\n",
    "import numpy as np\n",
    "import torch\n",
    "from datasets import load_dataset\n",
    "from transformers import (\n",
    "    AutoModelForSequenceClassification,\n",
    "    AutoTokenizer,\n",
    "    TrainingArguments,\n",
    "    Trainer\n",
    ")\n",
    "\n",
    "# Set the device\n",
    "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n",
    "print(f\"Using device: {device}\")\n",
    "\n",
    "# Define constants\n",
    "MODEL_NAME = \"distilbert-base-uncased\"  # Smaller model for faster training\n",
    "DATASET_NAME = \"glue\"\n",
    "DATASET_CONFIG = \"sst2\"  # Sentiment analysis task"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the pre-trained model\n",
    "model = AutoModelForSequenceClassification.from_pretrained(\n",
    "    MODEL_NAME, \n",
    "    num_labels=2,  # Binary classification\n",
    ").to(device)\n",
    "\n",
    "# Print model size\n",
    "total_params = sum(p.numel() for p in model.parameters())\n",
    "print(f\"Total parameters in base model: {total_params:,}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load tokenizer\n",
    "tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)\n",
    "\n",
    "# Load dataset \n",
    "dataset = load_dataset(DATASET_NAME, DATASET_CONFIG)\n",
    "print(f\"Dataset structure: {dataset}\")\n",
    "print(f\"Example from training set: {dataset['train'][0]}\")\n",
    "\n",
    "# To reduce computation, let's use a subset of the data\n",
    "train_dataset = dataset[\"train\"].select(range(1000))  # Use 1000 training examples\n",
    "eval_dataset = dataset[\"validation\"].select(range(200))  # Use 200 validation examples\n",
    "print(f\"Using {len(train_dataset)} training examples and {len(eval_dataset)} validation examples\")\n",
    "\n",
    "# Tokenize the dataset\n",
    "def preprocess_function(examples):\n",
    "    return tokenizer(\n",
    "        examples[\"sentence\"], \n",
    "        truncation=True, \n",
    "        padding=\"max_length\", \n",
    "        max_length=128\n",
    "    )\n",
    "\n",
    "# Apply preprocessing\n",
    "tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)\n",
    "tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define custom metrics functions without using sklearn\n",
    "def accuracy_score(y_true, y_pred):\n",
    "    \"\"\"Calculate accuracy score without using sklearn.\"\"\"\n",
    "    if len(y_true) != len(y_pred):\n",
    "        raise ValueError(\"Input arrays must have the same length\")\n",
    "    \n",
    "    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)\n",
    "    return correct / len(y_true)\n",
    "\n",
    "def f1_score(y_true, y_pred):\n",
    "    \"\"\"Calculate F1 score for binary classification without using sklearn.\"\"\"\n",
    "    if len(y_true) != len(y_pred):\n",
    "        raise ValueError(\"Input arrays must have the same length\")\n",
    "    \n",
    "    # Calculate true positives, false positives, false negatives\n",
    "    tp = sum(1 for true, pred in zip(y_true, y_pred) if true == 1 and pred == 1)\n",
    "    fp = sum(1 for true, pred in zip(y_true, y_pred) if true == 0 and pred == 1)\n",
    "    fn = sum(1 for true, pred in zip(y_true, y_pred) if true == 1 and pred == 0)\n",
    "    \n",
    "    # Calculate precision and recall\n",
    "    precision = tp / (tp + fp) if (tp + fp) > 0 else 0\n",
    "    recall = tp / (tp + fn) if (tp + fn) > 0 else 0\n",
    "    \n",
    "    # Calculate F1 score\n",
    "    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0\n",
    "    \n",
    "    return f1\n",
    "\n",
    "def compute_metrics(eval_pred):\n",
    "    \"\"\"Compute evaluation metrics for the model.\"\"\"\n",
    "    logits, labels = eval_pred\n",
    "    predictions = np.argmax(logits, axis=-1)\n",
    "    \n",
    "    return {\n",
    "        \"accuracy\": accuracy_score(labels, predictions),\n",
    "        \"f1\": f1_score(labels, predictions)\n",
    "    }\n",
    "\n",
    "# Set up training arguments for evaluation\n",
    "base_training_args = TrainingArguments(\n",
    "    output_dir=\"/tmp/base_eval\",\n",
    "    per_device_eval_batch_size=16,\n",
    "    report_to=\"none\"  # Disable reporting to avoid unnecessary dependencies\n",
    ")\n",
    "\n",
    "# Set up trainer for evaluation\n",
    "base_trainer = Trainer(\n",
    "    model=model,\n",
    "    args=base_training_args,\n",
    "    tokenizer=tokenizer,\n",
    "    compute_metrics=compute_metrics,\n",
    "    eval_dataset=tokenized_eval_dataset,\n",
    ")\n",
    "\n",
    "# Evaluate base model\n",
    "print(\"Evaluating base model...\")\n",
    "base_model_results = base_trainer.evaluate()\n",
    "print(\"Base model evaluation results:\")\n",
    "for key, value in base_model_results.items():\n",
    "    print(f\"  {key}: {value:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **Performing Parameter-Efficient Fine-Tuning**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import PEFT libraries\n",
    "from peft import get_peft_model, LoraConfig, TaskType\n",
    "\n",
    "# Configure PEFT (LoRA)\n",
    "peft_config = LoraConfig(\n",
    "    task_type=TaskType.SEQ_CLS,  # Sequence classification task\n",
    "    inference_mode=False,\n",
    "    r=8,                         # Rank of the low-rank decomposition\n",
    "    lora_alpha=32,               # Alpha parameter for LoRA scaling\n",
    "    lora_dropout=0.1,            # Dropout probability for LoRA layers\n",
    "    # Target the attention matrices in DistilBERT\n",
    "    target_modules=[\"q_lin\", \"v_lin\"]\n",
    ")\n",
    "\n",
    "# Apply PEFT config to the model\n",
    "peft_model = get_peft_model(model, peft_config)\n",
    "\n",
    "# Print trainable parameters information\n",
    "trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)\n",
    "print(f\"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)\")\n",
    "\n",
    "# Print model architecture\n",
    "print(peft_model)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Training arguments\n",
    "training_args = TrainingArguments(\n",
    "    output_dir=\"/tmp/checkpoints\",\n",
    "    learning_rate=1e-3,\n",
    "    per_device_train_batch_size=8,\n",
    "    per_device_eval_batch_size=16,\n",
    "    num_train_epochs=3,          # Train for 3 epochs\n",
    "    weight_decay=0.01,\n",
    "    evaluation_strategy=\"epoch\",\n",
    "    save_strategy=\"epoch\",\n",
    "    load_best_model_at_end=True,\n",
    "    report_to=\"none\"\n",
    ")\n",
    "\n",
    "# Initialize trainer\n",
    "trainer = Trainer(\n",
    "    model=peft_model,\n",
    "    args=training_args,\n",
    "    train_dataset=tokenized_train_dataset,\n",
    "    eval_dataset=tokenized_eval_dataset,\n",
    "    tokenizer=tokenizer,\n",
    "    compute_metrics=compute_metrics,\n",
    ")\n",
    "\n",
    "# Train the model\n",
    "print(\"Fine-tuning the model...\")\n",
    "train_results = trainer.train()\n",
    "\n",
    "# Print training results\n",
    "print(\"Training results:\")\n",
    "print(f\"  Training loss: {train_results.training_loss:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Saving the model\n",
    "peft_model_path = \"/tmp/peft_model_final\"\n",
    "trainer.save_model(peft_model_path)\n",
    "print(f\"Model saved to {peft_model_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **Performing Inference with a PEFT Model**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import PEFT model loading utilities\n",
    "from peft import PeftModel, PeftConfig\n",
    "\n",
    "# Load PEFT configuration\n",
    "peft_config = PeftConfig.from_pretrained(peft_model_path)\n",
    "\n",
    "# Load base model\n",
    "inference_model = AutoModelForSequenceClassification.from_pretrained(\n",
    "    peft_config.base_model_name_or_path,\n",
    "    num_labels=2\n",
    ")\n",
    "\n",
    "# Load PEFT model\n",
    "loaded_peft_model = PeftModel.from_pretrained(inference_model, peft_model_path)\n",
    "loaded_peft_model.to(device)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Set up trainer for evaluation\n",
    "eval_trainer = Trainer(\n",
    "    model=loaded_peft_model,\n",
    "    args=training_args,\n",
    "    tokenizer=tokenizer,\n",
    "    compute_metrics=compute_metrics,\n",
    "    eval_dataset=tokenized_eval_dataset,\n",
    ")\n",
    "\n",
    "# Evaluate fine-tuned model\n",
    "print(\"Evaluating fine-tuned model...\")\n",
    "fine_tuned_results = eval_trainer.evaluate()\n",
    "print(\"Fine-tuned model evaluation results:\")\n",
    "for key, value in fine_tuned_results.items():\n",
    "    print(f\"  {key}: {value:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare base and fine-tuned model results\n",
    "print(\"\\n=== Model Comparison ===\")\n",
    "print(\"Metric      | Base Model | Fine-tuned Model | Improvement\")\n",
    "print(\"------------|------------|-----------------|------------\")\n",
    "\n",
    "for metric in [\"eval_accuracy\", \"eval_f1\"]:\n",
    "    base_value = base_model_results.get(metric, 0)\n",
    "    fine_tuned_value = fine_tuned_results.get(metric, 0)\n",
    "    improvement = fine_tuned_value - base_value\n",
    "    \n",
    "    metric_name = metric.replace(\"eval_\", \"\")\n",
    "    print(f\"{metric_name:<11} | {base_value:.4f}     | {fine_tuned_value:.4f}          | {improvement:+.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Perform sample inference\n",
    "sample_examples = [\n",
    "    \"I absolutely loved this movie, it was fantastic!\",\n",
    "    \"The film was okay, nothing special.\",\n",
    "    \"This is the worst movie I've ever seen.\"\n",
    "]\n",
    "\n",
    "# Tokenize samples\n",
    "inputs = tokenizer(sample_examples, return_tensors=\"pt\", padding=True, truncation=True).to(device)\n",
    "\n",
    "# Perform inference with the fine-tuned model\n",
    "with torch.no_grad():\n",
    "    outputs = loaded_peft_model(**inputs)\n",
    "\n",
    "# Get predictions\n",
    "predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()\n",
    "\n",
    "# Display results\n",
    "print(\"\\nSample Inference:\")\n",
    "for i, (text, pred) in enumerate(zip(sample_examples, predictions)):\n",
    "    sentiment = \"Positive\" if pred == 1 else \"Negative\"\n",
    "    print(f\"Example {i+1}: \\\"{text}\\\"\")\n",
    "    print(f\"  Predicted sentiment: {sentiment}\")\n",
    "\n",
    "print(\"\\nProject completed successfully!\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}